# KdV Solitons

## Linearized KdV Equation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Fast FFT-based solution
def solve_kdv_fft(u0, x, t_values):
    """
    Solve linear KdV equation using FFT
    u0: initial condition
    x: spatial grid
    t_values: array of time values
    """
    N = len(x)
    L = x[-1] - x[0]
    dx = x[1] - x[0]

    # Wave numbers (correctly ordered for FFT)
    k = 2 * np.pi * np.fft.fftfreq(N, dx)

    # Fourier transform of initial condition
    u0_hat = np.fft.fft(u0)

    # Precompute solutions
    solutions = []
    for t in t_values:
        # Solution in Fourier space: û(k,t) = û₀(k) * exp(i k³ t)
        u_hat = u0_hat * np.exp(1j * k**3 * t)

        # Inverse transform back to physical space
        u = np.real(np.fft.ifft(u_hat))
        solutions.append(u)

    return np.array(solutions)

# Set up parameters
N = 512  # Number of grid points (power of 2 for efficiency)
L = 40.0  # Domain length
x = np.linspace(-L/2, L/2, N, endpoint=False)
dx = x[1] - x[0]

# Initial condition: Gaussian pulse
u0 = np.exp(-x**2)

# Time parameters
t_max = 3.0
num_frames = 100
t_values = np.linspace(0, t_max, num_frames)

print("Computing solution using FFT (this will be fast!)...")
solutions = solve_kdv_fft(u0, x, t_values)
print("Computation complete!")

# Create animation
fig, ax = plt.subplots(figsize=(6,4))
line, = ax.plot(x, solutions[0], 'b-', linewidth=3, label='u(x,t)')
ax.plot(x, u0, 'r--', linewidth=2, alpha=0.7, label='Initial condition (t=0)')

# Set up plot aesthetics
ax.set_xlim(-L/2, L/2)
ax.set_ylim(-0.5, 1.2)
ax.set_xlabel('Position (x)', fontsize=12)
ax.set_ylabel('Amplitude u(x,t)', fontsize=12)
ax.set_title('Dispersion in Linear KdV Equation: $u_t + u_{xxx} = 0$', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')

# Time annotation
time_text = ax.text(0.02, 0.95, 'Time: t = 0.00', transform=ax.transAxes, fontsize=12,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Info box
info_text = ax.text(0.02, 0.85, f'N = {N} points\nUsing FFT method',
                   transform=ax.transAxes, fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

def animate(i):
    """Update function for animation"""
    t = t_values[i]
    line.set_ydata(solutions[i])
    time_text.set_text(f'Time: t = {t:.2f}')

    # Change color based on time
    color_val = min(1.0, t/t_max)
    line.set_color((color_val, 0.3, 1-color_val))

    return line, time_text

# Create animation
ani = FuncAnimation(fig, animate, frames=num_frames, interval=50, blit=True)

plt.close(fig)  # Close to prevent double display

print("Animation ready!")
HTML(ani.to_jshtml())

## 1-Soliton Solution
+ The Korteweg-de Vries (KdV) equation is of the form:
$\frac{\partial u}{\partial t} + 6  u\frac{\partial u}{\partial x} + \frac{\partial^3 u}{\partial x^3} = 0 $

+ The analytical solution of above is:
$u(x,t) = \frac{c}{2} \cdot \operatorname{sech}^2 \left( \frac{\sqrt{c}}{2} (x - ct - \alpha_0) \right)$.

+ Choosing $c=4, \alpha_0=0$ the solution simplies to:
$u(x,t) = 2 \cdot \operatorname{sech}^2 (x - 4t)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button
from IPython.display import HTML, display

# Set up the data
x = np.linspace(-10, 10, 500)
num_frames = 100
t_values = np.linspace(0, 5, num_frames)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6,4))
plt.subplots_adjust(bottom=0.2)  # Make room for buttons

# Initialize the plot
line, = ax.plot([], [], lw=2, color='r')
ax.set_xlabel('x-axis')
ax.set_ylabel('u(x,t)')
ax.set_title('1-Soliton Solution of KdV Equation')
ax.set_xlim(-10, 10)
ax.set_ylim(0, 2)
ax.grid(True)

# Add equation annotation
#equation_text = r'$u(x,t) = 2 \cdot \mathrm{sech}^2(x - 4t)$'
#ax.text(0.02, 0.95, equation_text, transform=ax.transAxes, fontsize=12,
#       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Animation control variables
is_paused = False
current_frame = 0

# Animation function
def animate(frame):
    global current_frame
    current_frame = frame
    t = t_values[frame]
    u = 2 / np.cosh(x - 4*t)**2
    line.set_data(x, u)
    ax.set_title(f'1-Soliton (t = {t:.2f})')
    return line,

# Create the animation
ani = FuncAnimation(fig, animate, frames=num_frames, interval=50, blit=True, repeat=True)

# Button event handlers
def play(event):
    global is_paused
    is_paused = False
    ani.event_source.start()

def pause(event):
    global is_paused
    is_paused = True
    ani.event_source.stop()

# Display in Google Colab
plt.close(fig)  # Prevents duplicate display
HTML(ani.to_jshtml())